## 머신러닝 적용기 2

## 라이브러리 로드

In [44]:
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [45]:
RANDOM_STATE = 110

# 학습, 예측 데이터셋을 불러옵니다.
train = pd.read_csv("../../data/train_df_1012.csv")
test = pd.read_csv("../../data/test_df_1012.csv")
train.shape, test.shape

((54314, 4), (23405, 3))

In [46]:
train['분류'].value_counts().head(5)

지역          26850
경제:부동산       3447
사회:사건_사고     2545
경제:반도체       2309
사회:사회일반      1457
Name: 분류, dtype: int64

## 전처리를 위한 데이터 병합
* 학습, 예측 세트를 똑같이 전처리 해주기 위해 각각 같은 방법을 적용해 줄 수도 있지만
* 두 데이터를 병합해서 전처리를 해주고 다시 나눠주겠습니다.

## concat
<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg" width="500">

In [47]:
# 전처리를 위해 데이터 병합
df = pd.concat([train, test], axis=0)
train.shape, test.shape, df.shape

((54314, 4), (23405, 3), (77719, 4))

## 정답값 빈도수
* train 에서 제공하는 정답값의 빈도수

## 학습, 예측 데이터셋 분리
* 전처리가 되었다면 concat 으로 합쳐주었던 데이터프레임을 다시 학습과 예측 데이터셋으로 분리합니다.

In [48]:
# 정답값으로 사용할 topic_idx 를 변수에 담아 재사용 합니다.
label_name = "분류"

In [49]:
# label_name 을 기준으로 학습, 예측 세트를 분리합니다.
# label_name 이 있다면 학습 세트, 없다면 예측 세트가 됩니다.
# df_train
# df_test
df_train = df[df["분류"].notnull()].copy()
df_test = df[df["분류"].isnull()].copy()
df_train.shape, df_test.shape

((54314, 4), (23405, 4))

제목 + 키워드

In [50]:
import re

def clean_text(text):
    # 쉼표와 따옴표 제거
    text = re.sub(r'[,\']', '', text)
    # 한글 단어만 남기고 나머지 문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)
    return text

# '제목' 컬럼에 대해 처리
df_train['제목'] = df_train['제목'].apply(clean_text)
df_test['제목'] = df_test['제목'].apply(clean_text)

In [51]:
# 데이터 준비
df_train['키워드'] = df_train['제목'] + ' ' + df_train['키워드']
df_test['키워드'] = df_test['제목'] + ' ' + df_test['키워드']

In [25]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

# 데이터 확인
print(df_train.head())

# 텍스트와 레이블 분리
X = df_train['키워드']  # 키워드 컬럼
y = df_train['분류']  # 카테고리 컬럼

            ID        분류                                 제목  \
0  TRAIN_00000  문화:전시_공연  용인문화재단 인문학 콘서트 당신이 모르는 뮤지컬 이야기 개최   
1  TRAIN_00001        지역           용인 농촌테마파크 월 단체체험객 체험료 지원   
2  TRAIN_00002        지역       용인시 노후주택 에너지 성능 개선 신청 일까지 연장   
3  TRAIN_00003        지역       수원 용인 고양시특례시로 지정 도시경쟁력 증가 기대   
4  TRAIN_00004        국제   용인시 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할   

                                                 키워드  
0  용인문화재단 인문학 콘서트 당신이 모르는 뮤지컬 이야기 개최 용인문화재단 인문학 콘...  
1  용인 농촌테마파크 월 단체체험객 체험료 지원 농촌 테마파크 단체 체험객 체험료 체험...  
2  용인시 노후주택 에너지 성능 개선 신청 일까지 연장 노후 주택 에너지 성능 개선 신...  
3  수원 용인 고양시특례시로 지정 도시경쟁력 증가 기대 수원 고양시 특례시 지정 도시경...  
4  용인시 스페인 미국 국제명예자문관 위촉 대외홍보 지원 역할 스페인 미국 국제 명예 ...  


로지스틱

In [26]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

# 정규화
scaler = StandardScaler(with_mean=False)  # sparse matrix 이므로 with_mean=False
X_tfidf_scaled = scaler.fit_transform(X_tfidf)

# 데이터셋 나누기
X_train, X_test, y_train, y_test = train_test_split(X_tfidf_scaled, y, test_size=0.2, random_state=RANDOM_STATE)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score

# 모델 생성 및 학습
model = LogisticRegression(max_iter=5, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')

print(f'정확도: {accuracy}')
print('분류 리포트:')
print(report)
print(f'Macro F1-score: {macro_f1}')

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


정확도: 0.7988585105403664
분류 리포트:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.68      0.30      0.42        43
     IT_과학:과학       0.79      0.45      0.58        33
    IT_과학:모바일       0.74      0.62      0.68       108
     IT_과학:보안       1.00      0.33      0.50        21
IT_과학:인터넷_SNS       0.68      0.43      0.53        49
    IT_과학:콘텐츠       0.68      0.62      0.65        34
      경제:경제일반       0.56      0.32      0.41       109
    경제:금융_재테크       0.84      0.73      0.78        63
        경제:무역       0.90      0.79      0.84        68
       경제:반도체       0.79      0.90      0.84       452
       경제:부동산       0.85      0.87      0.86       686
     경제:산업_기업       0.77      0.58      0.66       159
    경제:서비스_쇼핑       0.76      0.65      0.70        86
        경제:유통       0.67      0.52      0.59       102
       경제:자동차       0.82      0.71      0.76       137
        경제:자원       0.71      0.50      0.59        34
     경제:증권_증시       0.86      0.

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [40]:
# %pip install gensim

Word2Vec 로지스틱

In [81]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 로드 및 전처리
X = train['키워드']
y = train['분류']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

# Word2Vec 모델 학습
sentences = [text.split() for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1) # sg=0: CBOW 모델, sg=1: Skip-gram 모델

# 텍스트를 벡터로 변환
def text_to_vector(text, model):
    words = text.split()
    word_vecs = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vecs, axis=0) if word_vecs else np.zeros(model.vector_size)

X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화
scaler = StandardScaler()
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# 로지스틱 회귀 모델 학습
lr_model = LogisticRegression(max_iter=5, random_state=RANDOM_STATE)
lr_model.fit(X_train_w2v_scaled, y_train)

# 예측 및 평가
y_valid_pred = lr_model.predict(X_valid_w2v_scaled)
print("Word2Vec/Logistic Regression")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"Macro F1-score: {macro_f1}")
print(classification_report(y_valid, y_valid_pred))

Word2Vec/Logistic Regression
Accuracy: 0.6838810641627543
Macro F1-score: 0.3596410051507864
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.00      0.00      0.00        43
     IT_과학:과학       0.00      0.00      0.00        33
    IT_과학:모바일       0.46      0.56      0.50       108
     IT_과학:보안       0.00      0.00      0.00        21
IT_과학:인터넷_SNS       0.00      0.00      0.00        49
    IT_과학:콘텐츠       0.00      0.00      0.00        34
      경제:경제일반       0.00      0.00      0.00       109
    경제:금융_재테크       0.60      0.56      0.58        63
        경제:무역       0.51      0.66      0.57        68
       경제:반도체       0.87      0.69      0.77       452
       경제:부동산       0.87      0.71      0.78       686
     경제:산업_기업       0.31      0.49      0.38       159
    경제:서비스_쇼핑       0.39      0.27      0.32        86
        경제:유통       0.29      0.57      0.38       102
       경제:자동차       0.50      0.58      0.54       137
        경제:자원       1.00  

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classif

2. CountVectorizer/RandomForest

In [ ]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score, classification_report
# from sklearn.preprocessing import StandardScaler

# # 데이터 확인
# print(df_train.head())

# # 텍스트와 레이블 분리
# X = df_train['키워드']  # 키워드 컬럼
# y = df_train['분류']  # 카테고리 컬럼

In [58]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

# CountVectorizer 벡터화
vectorizer = CountVectorizer()
X_train_cv = vectorizer.fit_transform(X_train)
X_valid_cv = vectorizer.transform(X_valid)

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf_model.fit(X_train_cv, y_train)

# 예측 및 평가
y_valid_pred = rf_model.predict(X_valid_cv)
accuracy = accuracy_score(y_valid, y_valid_pred)
report = classification_report(y_valid, y_valid_pred)
macro_f1 = f1_score(y_valid, y_valid_pred, average='macro')

print("CountVectorizer/RandomForest")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

CountVectorizer/RandomForest
Accuracy: 0.7782380557856946
Macro F1-score: 0.5519048177456197
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.76      0.30      0.43        43
     IT_과학:과학       1.00      0.30      0.47        33
    IT_과학:모바일       0.81      0.53      0.64       108
     IT_과학:보안       0.92      0.52      0.67        21
IT_과학:인터넷_SNS       0.57      0.43      0.49        49
    IT_과학:콘텐츠       0.81      0.50      0.62        34
      경제:경제일반       0.65      0.30      0.41       109
    경제:금융_재테크       0.93      0.63      0.75        63
        경제:무역       0.85      0.74      0.79        68
       경제:반도체       0.88      0.83      0.85       452
       경제:부동산       0.87      0.85      0.86       686
     경제:산업_기업       0.90      0.44      0.59       159
    경제:서비스_쇼핑       0.80      0.48      0.60        86
        경제:유통       0.78      0.37      0.50       102
       경제:자동차       0.86      0.59      0.70       137
   

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

3. Word2Vec/LightGBM

In [74]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

# Word2Vec 벡터화 (이미 학습된 모델 사용)
X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화 (이미 정규화된 데이터 사용)
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# y_train과 y_valid를 정수로 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# LightGBM 모델 학습
lgb_train = lgb.Dataset(X_train_w2v_scaled, label=y_train_encoded)
lgb_valid = lgb.Dataset(X_valid_w2v_scaled, label=y_valid_encoded, reference=lgb_train)

params = {
    'objective': 'multiclass',
    'num_class': len(set(y_train_encoded)),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31, # 하나의 트리가 가질 수 있는 최대 잎사귀 수
    'learning_rate': 0.05,
    'feature_fraction': 0.9, # 각 트리를 학습할 때 사용할 피처의 비율
    'verbose': -1,
    'random_state': RANDOM_STATE    
}

lgb_model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_valid], num_boost_round=10)

# 예측 및 평가
y_valid_pred = lgb_model.predict(X_valid_w2v_scaled, num_iteration=lgb_model.best_iteration)
y_valid_pred = np.argmax(y_valid_pred, axis=1)

accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
report = classification_report(y_valid_encoded, y_valid_pred, target_names=label_encoder.classes_)
macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')

print("Word2Vec/LightGBM")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

Word2Vec/LightGBM
Accuracy: 0.6863665654055049
Macro F1-score: 0.40077468024302726
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.25      0.04      0.07        48
     IT_과학:과학       0.45      0.35      0.39        43
    IT_과학:모바일       0.81      0.33      0.47       106
     IT_과학:보안       0.53      0.56      0.54        18
IT_과학:인터넷_SNS       0.43      0.25      0.32        48
    IT_과학:콘텐츠       0.52      0.38      0.44        32
      경제:경제일반       0.75      0.05      0.09       124
    경제:금융_재테크       0.68      0.55      0.61        65
        경제:무역       0.83      0.58      0.68        74
       경제:반도체       0.92      0.67      0.78       462
       경제:부동산       0.91      0.63      0.74       689
     경제:산업_기업       0.66      0.18      0.28       142
    경제:서비스_쇼핑       0.68      0.32      0.44        77
        경제:외환       0.07      0.50      0.12         2
        경제:유통       0.67      0.31      0.42       117
       경제:자동차

4. Word2Vec/RandomForest

In [75]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 분할 (클래스 비율 동일하게 유지)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Word2Vec 벡터화 (이미 학습된 모델 사용)
X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화 (이미 정규화된 데이터 사용)
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# y_train과 y_valid를 정수로 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)
rf_model.fit(X_train_w2v_scaled, y_train_encoded)

# 예측 및 평가
y_valid_pred = rf_model.predict(X_valid_w2v_scaled)

accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
report = classification_report(y_valid_encoded, y_valid_pred, target_names=label_encoder.classes_)
macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')

print("Word2Vec/RandomForest")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

Word2Vec/RandomForest
Accuracy: 0.8007916781736168
Macro F1-score: 0.6076433409984148
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.71      0.25      0.37        48
     IT_과학:과학       0.78      0.49      0.60        43
    IT_과학:모바일       0.72      0.49      0.58       106
     IT_과학:보안       1.00      0.61      0.76        18
IT_과학:인터넷_SNS       0.61      0.35      0.45        48
    IT_과학:콘텐츠       1.00      0.31      0.48        32
      경제:경제일반       0.73      0.29      0.42       124
    경제:금융_재테크       0.84      0.71      0.77        65
        경제:무역       0.93      0.68      0.78        74
       경제:반도체       0.89      0.87      0.88       462
       경제:부동산       0.85      0.87      0.86       689
     경제:산업_기업       0.74      0.52      0.61       142
    경제:서비스_쇼핑       0.78      0.65      0.71        77
        경제:외환       0.50      0.50      0.50         2
        경제:유통       0.82      0.60      0.69       117
       경제:

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [76]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 분할 (클래스 비율 동일하게 유지)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Word2Vec 벡터화 (이미 학습된 모델 사용)
X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화 (이미 정규화된 데이터 사용)
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# y_train과 y_valid를 정수로 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(n_estimators=500, random_state=RANDOM_STATE)
rf_model.fit(X_train_w2v_scaled, y_train_encoded)

# 예측 및 평가
y_valid_pred = rf_model.predict(X_valid_w2v_scaled)

accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
report = classification_report(y_valid_encoded, y_valid_pred, target_names=label_encoder.classes_)
macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')

print("Word2Vec/RandomForest")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

Word2Vec/RandomForest
Accuracy: 0.8020804565957839
Macro F1-score: 0.6155423380916296
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.67      0.25      0.36        48
     IT_과학:과학       0.77      0.47      0.58        43
    IT_과학:모바일       0.75      0.50      0.60       106
     IT_과학:보안       1.00      0.67      0.80        18
IT_과학:인터넷_SNS       0.58      0.40      0.47        48
    IT_과학:콘텐츠       0.83      0.31      0.45        32
      경제:경제일반       0.75      0.32      0.45       124
    경제:금융_재테크       0.81      0.66      0.73        65
        경제:무역       0.94      0.68      0.79        74
       경제:반도체       0.89      0.87      0.88       462
       경제:부동산       0.85      0.87      0.86       689
     경제:산업_기업       0.71      0.51      0.59       142
    경제:서비스_쇼핑       0.77      0.69      0.73        77
        경제:외환       1.00      0.50      0.67         2
        경제:유통       0.85      0.62      0.71       117
       경제:

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

5. Word2Vec/CatBoost

In [80]:
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 분할 (클래스 비율 동일하게 유지)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Word2Vec 벡터화 (이미 학습된 모델 사용)
X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화 (이미 정규화된 데이터 사용)
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# y_train과 y_valid를 정수로 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# CatBoost 모델 학습
catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.05, random_seed=RANDOM_STATE, verbose=0)
catboost_model.fit(X_train_w2v_scaled, y_train_encoded)

# 예측 및 평가
y_valid_pred = catboost_model.predict(X_valid_w2v_scaled)

accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
report = classification_report(y_valid_encoded, y_valid_pred, target_names=label_encoder.classes_)
macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')

print("Word2Vec/CatBoost")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

Word2Vec/CatBoost
Accuracy: 0.71996685998343
Macro F1-score: 0.3596410051507864
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.00      0.00      0.00        48
     IT_과학:과학       0.84      0.37      0.52        43
    IT_과학:모바일       0.55      0.29      0.38       106
     IT_과학:보안       1.00      0.06      0.11        18
IT_과학:인터넷_SNS       0.75      0.12      0.21        48
    IT_과학:콘텐츠       0.00      0.00      0.00        32
      경제:경제일반       1.00      0.04      0.08       124
    경제:금융_재테크       0.74      0.52      0.61        65
        경제:무역       0.81      0.58      0.68        74
       경제:반도체       0.79      0.80      0.79       462
       경제:부동산       0.80      0.80      0.80       689
     경제:산업_기업       0.55      0.39      0.45       142
    경제:서비스_쇼핑       0.59      0.34      0.43        77
        경제:외환       0.00      0.00      0.00         2
        경제:유통       0.62      0.38      0.47       117
       경제:자동차   

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [82]:
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 데이터 분할 (클래스 비율 동일하게 유지)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

# Word2Vec 벡터화 (이미 학습된 모델 사용)
X_train_w2v = np.array([text_to_vector(text, w2v_model) for text in X_train])
X_valid_w2v = np.array([text_to_vector(text, w2v_model) for text in X_valid])

# 정규화 (이미 정규화된 데이터 사용)
X_train_w2v_scaled = scaler.fit_transform(X_train_w2v)
X_valid_w2v_scaled = scaler.transform(X_valid_w2v)

# y_train과 y_valid를 정수로 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

# CatBoost 모델 학습
catboost_model = CatBoostClassifier(iterations=300, learning_rate=0.05, random_seed=RANDOM_STATE, verbose=0)
catboost_model.fit(X_train_w2v_scaled, y_train_encoded)

# 예측 및 평가
y_valid_pred = catboost_model.predict(X_valid_w2v_scaled)

accuracy = accuracy_score(y_valid_encoded, y_valid_pred)
report = classification_report(y_valid_encoded, y_valid_pred, target_names=label_encoder.classes_)
macro_f1 = f1_score(y_valid_encoded, y_valid_pred, average='macro')

print("Word2Vec/CatBoost")
print(f"Accuracy: {accuracy}")
print(f"Macro F1-score: {macro_f1}")
print("Classification Report:")
print(report)

Word2Vec/CatBoost
Accuracy: 0.7665469943846083
Macro F1-score: 0.502211778220137
Classification Report:
               precision    recall  f1-score   support

IT_과학:IT_과학일반       0.36      0.08      0.14        48
     IT_과학:과학       0.79      0.63      0.70        43
    IT_과학:모바일       0.63      0.48      0.55       106
     IT_과학:보안       1.00      0.28      0.43        18
IT_과학:인터넷_SNS       0.81      0.27      0.41        48
    IT_과학:콘텐츠       0.70      0.22      0.33        32
      경제:경제일반       1.00      0.08      0.15       124
    경제:금융_재테크       0.72      0.71      0.71        65
        경제:무역       0.88      0.69      0.77        74
       경제:반도체       0.83      0.84      0.83       462
       경제:부동산       0.83      0.83      0.83       689
     경제:산업_기업       0.53      0.45      0.49       142
    경제:서비스_쇼핑       0.69      0.57      0.62        77
        경제:외환       0.00      0.00      0.00         2
        경제:유통       0.64      0.53      0.58       117
       경제:자동차  

c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juneh\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

---

전체 데이터(train)로 학습 및 해당 모형으로 test 데이터 예측

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

RANDOM_STATE = 110

# 데이터 확인
print(train.head())

# 텍스트와 레이블 분리
X_train = train['키워드']  # 키워드 컬럼
y_train = train['분류']  # 카테고리 컬럼
X_test = test['키워드']  # 키워드 컬럼

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 정규화
scaler = StandardScaler(with_mean=False)  # sparse matrix 이므로 with_mean=False
X_train_tfidf_scaled = scaler.fit_transform(X_train_tfidf)
X_test_tfidf_scaled = scaler.transform(X_test_tfidf)

In [ ]:
# 모델 생성 및 학습
model = LogisticRegression(max_iter=1000, random_state=RANDOM_STATE)
model.fit(X_train_tfidf_scaled, y_train)

# 예측
y_test_pred = model.predict(X_test_tfidf_scaled)

# 결과 출력
print("예측 결과:")
print(y_test_pred)

# test 데이터셋에 예측 결과 추가
test['예측 분류'] = y_test_pred
print(test.head())

.